In [1]:
import numpy as np
import pandas as pd
from scipy.signal import welch 

In [2]:
def PSD(signal,sample_rate:int,npersseg):
    features=[]
    for k in range(32):
        points=signal.iloc[:,k]
        _,yf=welch(points.to_numpy(),sample_rate,nperseg=npersseg,window='hann')
        indices=int(npersseg/sample_rate*45)
        features.append(yf[:indices])
    return features

In [3]:
data_columns=['Fp1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3',
       'P7', 'PO3', 'O1', 'Oz', 'Pz', 'Fp2', 'AF4', 'Fz', 'F4', 'F8', 'FC6',
       'FC2', 'Cz', 'C4', 'T8', 'CP6', 'CP2', 'P4', 'P8', 'PO4', 'O2']

In [5]:
featured_data=[]
for i in range(1,33):
    subjects=[]
    for j in range(1,41):
        data=pd.read_csv(f'../EEG_processed/s{i}_{j}data.csv').transpose().iloc[385:,0:32]
        features=PSD(data,128,1024)
        subjects.append(features)
    featured_data.append(subjects)

In [6]:
arr=np.array(featured_data).reshape(460800,32)
data=pd.DataFrame(arr,columns=data_columns).to_csv()
with open(f'../datasets/PSD360.csv','a') as file:
    file.write(data)

In [10]:
data=pd.read_csv(f'../datasets/PSD360.csv').iloc[:,1:33]

In [11]:
featured_data=[]
for i in range(32):
    avg_data=[]
    segment=data.iloc[:,i].to_numpy()
    for j in range(0,460800,8):
        avg_data.append(np.mean(segment[j:j+8])) #for PSD(1280,32) change from 8 to 360
    featured_data.append(avg_data)

In [12]:
arr=np.array(featured_data).reshape(57600,32)
data=pd.DataFrame(arr,columns=data_columns).to_csv()
with open(f'../datasets/PSD45.csv','a') as file:
    file.write(data)